In [ ]:
from __future__ import print_function, division

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [ ]:
# list all files under the directory

path=os.getcwd()+ r'\raw\ODIR-5K\ODIR-5K_Training_Dataset'

image_folder=[]
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        image_folder.append( filename )

df = pd.read_excel("raw\ODIR-5K\ODIR-5K_Training_Annotations_V2.xlsx", index_col='ID')
df.head()

> <h3> Extracting AMD & Normal information from the Dataset </h3>

In [ ]:
def preprocess(df):
    df["left_amd"] = df["Left-Diagnostic Keywords"].apply(lambda x: 1 if "age-related" in x.lower() else 0)
        #df["left_drywet"] = df["Left-Diagnostic Keywords"].apply(lambda x: 1 if "wet age-related" in x.lower() else 0)

    df["right_amd"] = df["Right-Diagnostic Keywords"].apply(lambda x: 1 if "age-related" in x.lower() else 0)
        #df["right_drywet"] = df["Right-Diagnostic Keywords"].apply(lambda x: 1 if "wet age-related" in x.lower() else 0)
        
    df_left = df[[ "Left-Fundus", "left_amd"] ].rename({"Left-Fundus":'Fundus', "left_amd":'AMD'},axis=1)
    df_right = df[[ "Right-Fundus", "right_amd"] ].rename({"Right-Fundus":'Fundus', "right_amd":'AMD' },axis=1)
        
    return pd.concat([df_left, df_right])

df = preprocess(df)
df['Fundus'] = df['Fundus'].apply(lambda x: x.split('.')[0]+'.png')

In [ ]:
> <h3> Split Train/Test </h3>

In [ ]:
from sklearn.model_selection import train_test_split
df_aux, df_test = train_test_split(df, test_size=0.20, random_state=42)
df_train, df_valid = train_test_split(df_aux, test_size=0.20, random_state=42)

> <h3> Structure the dataset in order to use pytorch Image Folder class</h3>

In [ ]:
import shutil
from tqdm import tqdm
import os.path


def arrange_data(df, dataset_dir):
       
    amd = df.loc[df["AMD"] == 1, "Fundus"].values
    normal = df.loc[df["AMD"] == 0, "Fundus"].values
    
    #creating dir
    amd_path= dataset_dir+r'\amd'
    normal_path= dataset_dir+r'\normal'
    orig_path= os.getcwd()+ r'\raw\ODIR-5K\ODIR-5K_Training_Dataset'
    

    try:
        os.mkdir(amd_path)
        os.mkdir(normal_path)
    except OSError as error:
        print(error)
    
    #print(image_folder)
    #print("transfer AMD files from ODIR ")
    for img in tqdm(amd):
        original_image_path = os.path.join(orig_path,img)
        if img in image_folder:
            shutil.copy2(original_image_path, amd_path)

    #print("transfer Normal files from ODIR ")
    for img in tqdm(normal):
        original_image_path = os.path.join(orig_path,img)
        if img in image_folder:
            shutil.copy2(original_image_path, normal_path)

# RIADD

In [ ]:
# list all files under the directory

path=os.getcwd()+ r'\raw\RIADD\Training_set\Training'

image_folder=[]
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        image_folder.append( filename )

df = pd.read_csv("raw\RIADD\Training_set\RFMiD_Training_Labels.csv")
df.head()

In [ ]:
import shutil
from tqdm import tqdm
import os.path


def arrange_data(df, dataset_dir):
       
    amd = df.loc[df["ARMD"] == 1, "ID"].values
    normal = df.loc[df["ARMD"] == 0, "ID"].values
    
    #creating dir
    amd_path= dataset_dir+r'\amd'
    normal_path= dataset_dir+r'\normal'
    orig_path= os.getcwd()+ r'\raw\RIADD\Training_set\Training'
    

    try:
        os.mkdir(amd_path)
        os.mkdir(normal_path)
    except OSError as error:
        print(error)
    
    #print(image_folder)
    #print("transfer AMD files from ODIR ")
    for img in tqdm(amd):
        name_img= str(img)+'.png'
        original_image_path = os.path.join(orig_path,name_img)
        #print(original_image_path)
        if name_img in image_folder:
            #print(original_image_path)
            shutil.copy2(original_image_path, amd_path)

    #print("transfer Normal files from ODIR ")
    for img in tqdm(normal):
        name_img= str(img)+'.png'
        original_image_path = os.path.join(orig_path,name_img)
        if name_img in image_folder:
            shutil.copy2(original_image_path, normal_path)

In [ ]:
path= os.getcwd()+ r'\preprocess_1_folder\RIADD'

arrange_data(df, path)

100%|█████████████████████████████████████████████████████████████████████████████| 1820/1820 [00:08<00:00, 210.45it/s]


# folder to resize_align

In [ ]:
from utils.eyeq_preprocess import fundus_prep as prep
import glob
import os
import cv2 as cv
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
def process(image_list, save_path):
    
    for image_path in image_list:
        dst_image = os.path.splitext(image_path.split("\\")[-1])[0]+'.png'
        dst_path = os.path.join(save_path, dst_image)
        if os.path.exists(dst_path):
            print('continue...')
            continue
        try:
            img = prep.imread(image_path)
            r_img, borders, mask = prep.process_without_gb(img)
            r_img = cv.resize(r_img, (800, 800))
            prep.imwrite(dst_path, r_img)
            # mask = cv.resize(mask, (800, 800))
            # prep.imwrite(os.path.join('./original_mask', dst_image), mask)
        except:
            print(image_path)
            continue

# ODIR-5k

In [ ]:
path=os.getcwd()+ r'\preprocess_2_crop_resize_align\ODIR-5K'
path_amd = path+r'\AMD'
path_non = path+r'\Non-AMD'


path_orig= os.getcwd()+ r'\preprocess_1_folder\ODIR-5K'
path_orig_amd = path_orig+r'\AMD'
path_orig_non = path_orig+r'\Non-AMD'

image_list = glob.glob(os.path.join(path_orig_amd, '*.jpg'))
save_path = prep.fold_dir(path_amd)
process(image_list, save_path)

image_list = glob.glob(os.path.join(path_orig_non, '*.jpg'))
save_path = prep.fold_dir(path_non)
process(image_list, save_path)

# Baidu_Challenge

In [ ]:
path=os.getcwd()+ r'\preprocess_2_crop_resize_align\Baidu_Challenge'
path_amd = path+r'\AMD'
path_non = path+r'\Non-AMD'


path_orig= os.getcwd()+ r'\preprocess_1_folder\Baidu_Challenge'
path_orig_amd = path_orig+r'\AMD'
path_orig_non = path_orig+r'\Non-AMD'

image_list = glob.glob(os.path.join(path_orig_amd, '*.jpg'))
save_path = prep.fold_dir(path_amd)
process(image_list, save_path)

image_list = glob.glob(os.path.join(path_orig_non, '*.jpg'))
save_path = prep.fold_dir(path_non)
process(image_list, save_path)

# RIADD

In [ ]:
path=os.getcwd()+ r'\preprocess_2_crop_resize_align\RIADD'
path_amd = path+r'\AMD'
path_non = path+r'\Non-AMD'


path_orig= os.getcwd()+ r'\preprocess_1_folder\RIADD'
path_orig_amd = path_orig+r'\AMD'
path_orig_non = path_orig+r'\Non-AMD'

image_list = glob.glob(os.path.join(path_orig_amd, '*.png'))
save_path = prep.fold_dir(path_amd)
process(image_list, save_path)

image_list = glob.glob(os.path.join(path_orig_non, '*.png'))
save_path = prep.fold_dir(path_non)
process(image_list, save_path)

# resize_align to Quality assesment

In [ ]:
import os
import argparse
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import time
from utils.eyeq_model.progress.bar import Bar

import torchvision.transforms as transforms
from utils.eyeq_model.dataloader.EyeQ_loader import DatasetGenerator

import pandas as pd
from utils.eyeq_model.networks.densenet_mcf import dense121_mcs
import shutil
from tqdm import tqdm

In [ ]:
def prop_quality(test_images_dir, path_dest ):    
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    np.random.seed(0)

    #data_root = '../Kaggle_DR_dataset/'

    # Setting parameters
    parser = argparse.ArgumentParser(description='EyeQ_dense121')
    parser.add_argument('--model_dir', type=str, default='./utils/eyeq_model/result')
    parser.add_argument('--pre_model', type=str, default='DenseNet121_v3_v1')
    #parser.add_argument('--save_model', type=str, default='DenseNet121_v3_v1')

    parser.add_argument('--crop_size', type=int, default=224)
    parser.add_argument('--label_idx', type=list, default=['Good', 'Usable', 'Reject'])

    parser.add_argument('--n_classes', type=int, default=3)
    # Optimization options
    parser.add_argument('--epochs', default=20, type=int)
    parser.add_argument('--batch-size', default=2, type=int)
    parser.add_argument('--lr', default=0.01, type=float)
    parser.add_argument('--loss_w', default=[0.1, 0.1, 0.1, 0.1, 0.6], type=list)

    args, unknown = parser.parse_known_args()


    # options
    cudnn.benchmark = True

    model = dense121_mcs(n_class=args.n_classes)

    if args.pre_model is not None:
        loaded_model = torch.load(os.path.join(args.model_dir, args.pre_model + '.tar'))
        model.load_state_dict(loaded_model['state_dict'])

    model.to(device)

    criterion = torch.nn.BCELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)

    print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))


    transform_list_val1 = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
        ])

    transformList2 = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406],
                                [0.229, 0.224, 0.225])
        ])


    data_test = DatasetGenerator(data_dir=test_images_dir,list_file=None, transform1=transform_list_val1,
                                transform2=transformList2, n_class=args.n_classes, set_name='test')
    test_loader = torch.utils.data.DataLoader(dataset=data_test, batch_size=args.batch_size,
                                            shuffle=False, num_workers=0, pin_memory=True)


    # Testing
    outPRED_mcs = torch.FloatTensor().cuda()
    img_name_mcs = []
    model.eval()
    iters_per_epoch = len(test_loader)
    bar = Bar('Processing {}'.format('inference'), max=len(test_loader))
    bar.check_tty = False
    for epochID, (imagesA, imagesB, imagesC, image_name) in enumerate(test_loader):
        imagesA = imagesA.cuda()
        imagesB = imagesB.cuda()
        imagesC = imagesC.cuda()

        begin_time = time.time()
        _, _, _, _, result_mcs = model(imagesA, imagesB, imagesC)
        outPRED_mcs = torch.cat((outPRED_mcs, result_mcs.data), 0)
        for i in image_name:
            img_name_mcs.append(i)

        batch_time = time.time() - begin_time
        bar.suffix = '{} / {} | Time: {batch_time:.4f}'.format(epochID + 1, len(test_loader),
                                                            batch_time=batch_time * (iters_per_epoch - epochID) / 60)
        bar.next()
    bar.finish()

    pred= torch.argmax(outPRED_mcs,1).cpu().numpy()
    img_name_mcs= np.asarray(img_name_mcs)
    print(outPRED_mcs)

    path=path_dest

    try:
        os.mkdir(path)
    except OSError as error:
        print(error)

    goods = img_name_mcs[pred!=2]
    for img in tqdm( goods ):
        original_image_path = img
        shutil.copy2(original_image_path, path)

# Baidu_Challenge

In [ ]:
test_images_dir = os.getcwd()+ r'\preprocess_2_crop_resize_align\Baidu_Challenge'
path_amd = test_images_dir+r'\AMD'
path_non = test_images_dir+r'\Non-AMD'


path_dest= os.getcwd()+ r'\preprocess_3_quality\Baidu_Challenge'
path_dest_amd = path_dest+r'\AMD'
path_dest_non = path_dest+r'\Non-AMD'


#prop_quality(path_amd, path_dest_amd )
prop_quality(path_non, path_dest_non )

# ODIR-5K

In [ ]:
test_images_dir = os.getcwd()+ r'\preprocess_2_crop_resize_align\ODIR-5k'
path_amd = test_images_dir+r'\AMD'
path_non = test_images_dir+r'\Non-AMD'


path_dest= os.getcwd()+ r'\preprocess_3_quality\ODIR-5k'
path_dest_amd = path_dest+r'\AMD'
path_dest_non = path_dest+r'\Non-AMD'


prop_quality(path_amd, path_dest_amd )
prop_quality(path_non, path_dest_non )

# RIADD

In [ ]:
test_images_dir = os.getcwd()+ r'\preprocess_2_crop_resize_align\RIADD'
path_amd = test_images_dir+r'\AMD'
path_non = test_images_dir+r'\Non-AMD'


path_dest= os.getcwd()+ r'\preprocess_3_quality\RIADD'
path_dest_amd = path_dest+r'\AMD'
path_dest_non = path_dest+r'\Non-AMD'


prop_quality(path_amd, path_dest_amd )
prop_quality(path_non, path_dest_non )

# Quality assesment to train and test

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import pandas as pd
from PIL import Image

plt.ion()   # interactive mode

In [ ]:
data_transforms = {
        'valid': transforms.Compose([
            transforms.Resize(390),
            transforms.CenterCrop(256),
            transforms.ToTensor()
        ]),
}

image_datasets={}

    
image_datasets['Baidu-A'] = torchvision.datasets.ImageFolder(root=r'preprocess_3_quality\Baidu_Challenge\AMD', transform=data_transforms['valid'])
image_datasets['Baidu-N'] = torchvision.datasets.ImageFolder(root=r'preprocess_3_quality\Baidu_Challenge\Non-AMD',transform=data_transforms['valid'])
image_datasets['Odir-A'] = torchvision.datasets.ImageFolder(root=r'preprocess_3_quality\ODIR-5K\AMD', transform=data_transforms['valid'])
image_datasets['Odir-N'] = torchvision.datasets.ImageFolder(root=r'preprocess_3_quality\ODIR-5K\Non-AMD', transform=data_transforms['valid'])
image_datasets['Riadd-A'] = torchvision.datasets.ImageFolder(root=r'preprocess_3_quality\RIADD\AMD', transform=data_transforms['valid'])
image_datasets['Riadd-N'] = torchvision.datasets.ImageFolder(root=r'preprocess_3_quality\RIADD\Non-AMD', transform=data_transforms['valid'])

In [ ]:
from torchvision.utils import save_image

baidu_idx = int(len(image_datasets['Baidu-A']) *0.28)
odir_idx = int(len(image_datasets['Odir-A']) *0.28)
riadd_idx = int(len(image_datasets['Riadd-A']) *0.28)

count=0

for i in range(0,baidu_idx):
    img=image_datasets['Baidu-A'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/test/AMD/{}.png'.format(count) )
    count=count+1

for i in range(baidu_idx, len(image_datasets['Baidu-A']) ):
    img=image_datasets['Baidu-A'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/train/AMD/{}.png'.format(count) )
    count=count+1
    

for i in range(0,odir_idx):
    img=image_datasets['Odir-A'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/test/AMD/{}.png'.format(count) )
    count=count+1

for i in range(odir_idx, len(image_datasets['Odir-A']) ):
    img=image_datasets['Odir-A'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/train/AMD/{}.png'.format(count) )
    count=count+1

    

for i in range(0,riadd_idx):
    img=image_datasets['Riadd-A'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/test/AMD/{}.png'.format(count) )
    count=count+1

for i in range(riadd_idx, len(image_datasets['Riadd-A']) ):
    img=image_datasets['Riadd-A'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/train/AMD/{}.png'.format(count) )
    count=count+1

In [ ]:
#create test

totalFiles=0
for base, dirs, files in os.walk(r'preprocess_4_create_crop/data/test/AMD'):
    print('Searching in : ',base)
    for Files in files:
        totalFiles += 1

In [ ]:

totalFiles

In [ ]:
from torchvision.utils import save_image

count=0

for i in range(0,baidu_idx):
    img=image_datasets['Baidu-N'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/test/Non-AMD/{}.png'.format(count) )
    count=count+1

for i in range(baidu_idx, len(image_datasets['Baidu-N']) ):
    img=image_datasets['Baidu-N'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/train/Non-AMD/{}.png'.format(count) )
    count=count+1
    

for i in range(0,odir_idx):
    img=image_datasets['Odir-N'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/test/Non-AMD/{}.png'.format(count) )
    count=count+1

for i in range(odir_idx, len(image_datasets['Odir-N']) ):
    img=image_datasets['Odir-N'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/train/Non-AMD/{}.png'.format(count) )
    count=count+1

    

for i in range(0,riadd_idx):
    img=image_datasets['Riadd-N'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/test/Non-AMD/{}.png'.format(count) )
    count=count+1

for i in range(riadd_idx, len(image_datasets['Riadd-N']) ):
    img=image_datasets['Riadd-N'][i][0]
    save_image(img, 'preprocess_4_create_crop/data/train/Non-AMD/{}.png'.format(count) )
    count=count+1

# To Conditional Stylegan Format

In [ ]:
from __future__ import print_function, division

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [ ]:
path=os.getcwd()+r'\preprocess_4_create_crop\data\train'

dataset={}
dataset['labels'] = []

new_name={}

i=-2
idd=0

for dirpath, _, files in os.walk(path):
    i=i+1
    for file_name in files:
        idd= idd+1
        new_name[os.path.join(dirpath, file_name)] = str(idd)+'.png'
        dataset['labels'].append([str(idd)+'.png',i])

In [ ]:
import shutil

dataset_dir = os.getcwd() + r'\preprocess_5_stylegan_format\train'

try:
    os.mkdir(dataset_dir)
except OSError as error:
    print(error)
    
for p in new_name.keys():
    shutil.copy2(p, os.path.join( dataset_dir, new_name[p] ) )

In [ ]:
import json

json.dumps(dataset)

with open(dataset_dir+r'\dataset.json', 'w') as outfile:
    json.dump(dataset, outfile)